In [1]:
import numpy as np 
import pandas as pd 
import sys
import pickle as pkl
import pickle
import joblib
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import keras
import os
from tpot import TPOTClassifier
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
import sklearn.metrics
import itertools
from math import ceil 
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler,LabelEncoder

Using TensorFlow backend.


In [2]:
# load  preprocessed dataset
data=pd.read_csv("Train.csv")

data_new=data.as_matrix()

X=data_new[:,:-1]
Y=data_new[:,-1]

#train-test split
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split( X, Y, test_size=0.30,random_state=7)

C:\Users\User\Anaconda3\envs\tf_gpu\lib\site-packages\ipykernel_launcher.py:6: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  


In [3]:
#limit the GPU memory this program is using
config  = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.6
keras.backend.set_session(tf.Session(config = config))

In [7]:
tpot = TPOTClassifier(verbosity=3, 
                      scoring="balanced_accuracy", 
                      random_state=23, 
                      periodic_checkpoint_folder="tpot_results", 
                      n_jobs=1, 
                      generations=5, 
                      population_size=50)

In [8]:
tpot.fit(x_train,y_train)


30 operators have been imported by TPOT.


Skipped pipeline #6 due to time out. Continuing to the next pipeline.
Skipped pipeline #8 due to time out. Continuing to the next pipeline.
Skipped pipeline #10 due to time out. Continuing to the next pipeline.
Skipped pipeline #16 due to time out. Continuing to the next pipeline.
Skipped pipeline #19 due to time out. Continuing to the next pipeline.
Skipped pipeline #21 due to time out. Continuing to the next pipeline.
Skipped pipeline #24 due to time out. Continuing to the next pipeline.
Skipped pipeline #26 due to time out. Continuing to the next pipeline.
Skipped pipeline #28 due to time out. Continuing to the next pipeline.
Skipped pipeline #31 due to time out. Continuing to the next pipeline.
Skipped pipeline #33 due to time out. Continuing to the next pipeline.
Skipped pipeline #35 due to time out. Continuing to the next pipeline.
Skipped pipeline #37 due to time out. Continuing to the next pipeline.
Skipped pipeline #39 due to time out. Continuing to the next pipeline.
Skipped 

Skipped pipeline #238 due to time out. Continuing to the next pipeline.
Skipped pipeline #240 due to time out. Continuing to the next pipeline.
Skipped pipeline #243 due to time out. Continuing to the next pipeline.
Skipped pipeline #247 due to time out. Continuing to the next pipeline.
Skipped pipeline #253 due to time out. Continuing to the next pipeline.
Skipped pipeline #258 due to time out. Continuing to the next pipeline.
Skipped pipeline #262 due to time out. Continuing to the next pipeline.
Skipped pipeline #264 due to time out. Continuing to the next pipeline.
Skipped pipeline #266 due to time out. Continuing to the next pipeline.
Skipped pipeline #270 due to time out. Continuing to the next pipeline.
Skipped pipeline #276 due to time out. Continuing to the next pipeline.
Generation 3 - Current Pareto front scores:
-1	0.9975829925234446	DecisionTreeClassifier(input_matrix, DecisionTreeClassifier__criterion=entropy, DecisionTreeClassifier__max_depth=10, DecisionTreeClassifier__

TPOTClassifier(config_dict=None, crossover_rate=0.1, cv=5,
               disable_update_check=False, early_stop=None, generations=5,
               max_eval_time_mins=5, max_time_mins=None, memory=None,
               mutation_rate=0.9, n_jobs=1, offspring_size=None,
               periodic_checkpoint_folder='tpot_results', population_size=50,
               random_state=23, scoring='balanced_accuracy', subsample=1.0,
               template=None, use_dask=False, verbosity=3, warm_start=False)

In [9]:
print(tpot.score(x_test, y_test))
tpot.export('tpot_pipeline.py')

0.9991391395981906


In [ ]:
##########training the exported tpot_pipeline.py file#############

In [16]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline, make_union
from sklearn.tree import DecisionTreeClassifier
from tpot.builtins import StackingEstimator

tpot_data = pd.read_csv('train.csv')
features = tpot_data.drop('Label', axis=1).values
training_features, testing_features, training_target, testing_target = \
            train_test_split(features, tpot_data['Label'].values, random_state=23)

# Average CV score on the training set was:0.9988887275364388
exported_pipeline = make_pipeline(
    StackingEstimator(estimator=DecisionTreeClassifier(criterion="entropy", max_depth=10, min_samples_leaf=4, min_samples_split=20)),
    DecisionTreeClassifier(criterion="entropy", max_depth=10, min_samples_leaf=10, min_samples_split=16)
)




In [17]:
exported_pipeline.fit(training_features, training_target)


In [18]:
print("Validation Accuracy",exported_pipeline.score(testing_features, testing_target)*100)
results = exported_pipeline.predict(testing_features)
print("Classification Report:-\n",classification_report(testing_target,results))

Validation Accuracy 99.91672364857381
Classification Report:-
               precision    recall  f1-score   support

           0       1.00      1.00      1.00    166891
           1       1.00      1.00      1.00     94888

    accuracy                           1.00    261779
   macro avg       1.00      1.00      1.00    261779
weighted avg       1.00      1.00      1.00    261779



In [20]:
# Compute confusion matrix
from sklearn.metrics import classification_report
from sklearn.metrics import  confusion_matrix
from sklearn.model_selection import train_test_split,cross_val_score,ShuffleSplit
cnf_matrix1 = confusion_matrix(testing_target,results)

print("Confusion matrix:")
print(cnf_matrix1)
tn = cnf_matrix1[0][0]
fp = cnf_matrix1[0][1]
fn = cnf_matrix1[1][0]
tp = cnf_matrix1[1][1]
print("\nTotal number of true positives(Attack identified as Attack))", tp)
print("Total number of false negatives (Attack identified as Benign)",fn)
print("Total number of false positives(Benign identified as Attack)",fp)
print("Total number of true negatives (Benign identified as Benign)",tn)

rp=tp+fn
rn=fp+tn
pp=tp+fp
pn=tn+fn

print("Real positives:",rp)
print("Real negatives:",rn)
print("Predicted positives:",pp)
print("Predicted negatives:",pn,"\n")

Confusion matrix:
[[166756    135]
 [    83  94805]]

Total number of true positives(Attack identified as Attack)) 94805
Total number of false negatives (Attack identified as Benign) 83
Total number of false positives(Benign identified as Attack) 135
Total number of true negatives (Benign identified as Benign) 166756
Real positives: 94888
Real negatives: 166891
Predicted positives: 94940
Predicted negatives: 166839 



In [19]:
pickle.dump(exported_pipeline, open("TPOT.dat", "wb"))

In [ ]:
##############prediction on synthesized test attacks##############

In [21]:
#predict on synthesized Slow httptest DoS attack

test_data=pd.read_csv('ShttpAA1.csv')
test_data_new=test_data.as_matrix()

X_test1=test_data_new[:,:-1] 
Y_test1=test_data_new[:,-1]

#predict on synthesized Slowloris DoS attack

test_data2=pd.read_csv('SlowAA1.csv')
test_data_new2=test_data2.as_matrix()

X_test2=test_data_new2[:,:-1]
Y_test2=test_data_new2[:,-1]

C:\Users\User\Anaconda3\envs\tf_gpu\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  after removing the cwd from sys.path.
C:\Users\User\Anaconda3\envs\tf_gpu\lib\site-packages\ipykernel_launcher.py:13: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  del sys.path[0]


In [22]:
# load model from file
model2 = pickle.load(open("TPOT.dat", "rb"))

Y_pred1 = model2.predict(X_test1)
print("Testing Accuracy for Slow httptest= ", model2.score(X_test1,Y_test1)*100)
print("Classification Report for Slow httptest:-\n",classification_report(Y_test1,Y_pred1))

Testing Accuracy for Slow httptest=  69.13220733838091
Classification Report for Slow httptest:-
               precision    recall  f1-score   support

         0.0       0.32      1.00      0.48       244
         1.0       1.00      0.64      0.78      1473

    accuracy                           0.69      1717
   macro avg       0.66      0.82      0.63      1717
weighted avg       0.90      0.69      0.74      1717



In [23]:
Y_pred2 = model2.predict(X_test2)
print("Testing Accuracy for Slowloris= ", model2.score(X_test2,Y_test2)*100)
print("Classification Report for Slowloris:-\n",classification_report(Y_test2,Y_pred2))

Testing Accuracy for Slowloris=  100.0
Classification Report for Slowloris:-
               precision    recall  f1-score   support

         0.0       1.00      1.00      1.00       244
         1.0       1.00      1.00      1.00       268

    accuracy                           1.00       512
   macro avg       1.00      1.00      1.00       512
weighted avg       1.00      1.00      1.00       512



In [24]:
# Compute confusion matrix for Slow httptest
from sklearn.metrics import classification_report
from sklearn.metrics import  confusion_matrix
from sklearn.model_selection import train_test_split,cross_val_score,ShuffleSplit
cm1 = confusion_matrix(Y_test1, Y_pred1)
print("Confusion matrix for Slow httptest:")
print(cm1)
tn = cm1[0][0]
fp = cm1[0][1]
fn = cm1[1][0]
tp = cm1[1][1]
print("\nTotal number of true positives(Attack identified as Attack))", tp)
print("Total number of false negatives (Attack identified as Benign)",fn)
print("Total number of false positives(Benign identified as Attack)",fp)
print("Total number of true negatives (Benign identified as Benign)",tn)

rp=tp+fn
rn=fp+tn
pp=tp+fp
pn=tn+fn

print("Real positives:",rp)
print("Real negatives:",rn)
print("Predicted positives:",pp)
print("Predicted negatives:",pn,"\n")

Confusion matrix for Slow httptest:
[[244   0]
 [530 943]]

Total number of true positives(Attack identified as Attack)) 943
Total number of false negatives (Attack identified as Benign) 530
Total number of false positives(Benign identified as Attack) 0
Total number of true negatives (Benign identified as Benign) 244
Real positives: 1473
Real negatives: 244
Predicted positives: 943
Predicted negatives: 774 



In [25]:
# Compute confusion matrix for Slowloris
from sklearn.metrics import classification_report
from sklearn.metrics import  confusion_matrix
from sklearn.model_selection import train_test_split,cross_val_score,ShuffleSplit
cm2 = confusion_matrix(Y_test2, Y_pred2)
print("Confusion matrix for Slowloris:")
print(cm2)
tn = cm2[0][0]
fp = cm2[0][1]
fn = cm2[1][0]
tp = cm2[1][1]
print("\nTotal number of true positives(Attack identified as Attack))", tp)
print("Total number of false negatives (Attack identified as Benign)",fn)
print("Total number of false positives(Benign identified as Attack)",fp)
print("Total number of true negatives (Benign identified as Benign)",tn)

rp=tp+fn
rn=fp+tn
pp=tp+fp
pn=tn+fn

print("Real positives:",rp)
print("Real negatives:",rn)
print("Predicted positives:",pp)
print("Predicted negatives:",pn,"\n")

Confusion matrix for Slowloris:
[[244   0]
 [  0 268]]

Total number of true positives(Attack identified as Attack)) 268
Total number of false negatives (Attack identified as Benign) 0
Total number of false positives(Benign identified as Attack) 0
Total number of true negatives (Benign identified as Benign) 244
Real positives: 268
Real negatives: 244
Predicted positives: 268
Predicted negatives: 244 

